In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import librosa
from tqdm import tqdm_notebook as tqdm
import json
import pandas as pd
import pickle
import os
import h5py
import collections as cl
from ast import literal_eval
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
import random
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from deem import deem

In [2]:
param_grid = {'LR_param': {'C':[10**k for k in range(-10, 6, 2)]}, 'scoring': 'roc_auc', 'cv': 3, 'random_state': 42}

with open("openmic_classmap_10.json", "r") as f: # only consider 10 classes of Openmic dataset
    class_map = json.load(f)
    
# use a dict to align the classes between Openmic dataset (key) and Irmas dataset (val)
with open("class_align.json", "r") as f: # only consider 10 classes of Openmic dataset
    class_align = json.load(f)

with open("genre_map.json", "r") as f: # only consider 10 classes of Openmic dataset
    genre_map = json.load(f)
aligned_genre = list(genre_map)[:-1]

with open("instrument_map.json", "r") as f: # only consider 10 classes of Openmic dataset
    instrument_map = json.load(f)

## VGGish

In [3]:
embedding = 'vggish'
debias_method = ''

deb = deem(embedding = embedding, debias_method = debias_method, feature_dir='./embeddings.h5', 
           instrument_map=instrument_map, genre_map=genre_map, param_grid=param_grid, class_align=class_align)

irmas_feature = deb.load_irmas()
openmic_feature = deb.load_openmic()

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

(13410, 128) (13410,)
(6705,)


100%|██████████| 6705/6705 [00:01<00:00, 4625.14it/s]


(6705, 128) (6705,)
(180000, 128) (180000,)


100%|██████████| 20000/20000 [00:57<00:00, 350.03it/s]


(20000, 128) (20000,)
(20000, 20) (20000, 20) (20000,)


...

100%|██████████| 10/10 [00:16<00:00,  1.68s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [00:16<00:00,  1.65s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [00:17<00:00,  1.73s/it]


In [4]:
debias_method = '-lda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

deb.result_all.to_csv('results/result_' + embedding + debias_method + '.csv', index=False)
result_all = deb.result_all

Train on irmas, test on irmas


100%|██████████| 10/10 [00:18<00:00,  1.86s/it]


Train on irmas, test on openmic


100%|██████████| 10/10 [00:16<00:00,  1.60s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [00:13<00:00,  1.37s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [00:14<00:00,  1.45s/it]


## OpenL3

In [5]:
embedding = 'openl3'
debias_method = ''

deb = deem(embedding = embedding, debias_method = debias_method, feature_dir='./embeddings.h5', 
           instrument_map=instrument_map, genre_map=genre_map, param_grid=param_grid, class_align=class_align)

irmas_feature = deb.load_irmas()
openmic_feature = deb.load_openmic()

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

(167625, 512) (167625,)
(6705,)


100%|██████████| 6705/6705 [00:17<00:00, 376.70it/s]


(6705, 512) (6705,)
(1900000, 512) (1900000,)


100%|██████████| 20000/20000 [10:00<00:00, 33.32it/s]


(20000, 512) (20000,)
(20000, 20) (20000, 20) (20000,)


...

100%|██████████| 10/10 [00:11<00:00,  1.17s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [00:12<00:00,  1.29s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [00:13<00:00,  1.34s/it]


In [6]:
debias_method = '-lda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

deb.result_all.to_csv('results/result_' + embedding + debias_method + '.csv', index=False)
result_all = result_all.append(deb.result_all)

Train on irmas, test on irmas


100%|██████████| 10/10 [00:12<00:00,  1.25s/it]


Train on irmas, test on openmic


100%|██████████| 10/10 [00:12<00:00,  1.22s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [00:13<00:00,  1.39s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [00:14<00:00,  1.45s/it]


## YAMNet

In [7]:
embedding = 'yamnet'
debias_method = ''

deb = deem(embedding = embedding, debias_method = debias_method, feature_dir='./embeddings.h5', 
           instrument_map=instrument_map, genre_map=genre_map, param_grid=param_grid, class_align=class_align)

irmas_feature = deb.load_irmas()
openmic_feature = deb.load_openmic()

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

(33525, 1024) (33525,)
(6705,)


100%|██████████| 6705/6705 [00:03<00:00, 1788.60it/s]


(6705, 1024) (6705,)
(380000, 1024) (380000,)


100%|██████████| 20000/20000 [02:03<00:00, 161.34it/s]


(20000, 1024) (20000,)
(20000, 20) (20000, 20) (20000,)


...

100%|██████████| 10/10 [00:11<00:00,  1.19s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [00:15<00:00,  1.58s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [00:16<00:00,  1.62s/it]


In [8]:
debias_method = '-lda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

deb.result_all.to_csv('results/result_' + embedding + debias_method + '.csv', index=False)
result_all = result_all.append(deb.result_all)

Train on irmas, test on irmas


100%|██████████| 10/10 [00:22<00:00,  2.28s/it]


Train on irmas, test on openmic


100%|██████████| 10/10 [00:22<00:00,  2.20s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [00:26<00:00,  2.66s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [00:26<00:00,  2.67s/it]


In [9]:
result_all.to_csv('results/result_all_lda.csv', index=False)

In [10]:
result_all

,instrument,train_set,test_set,precision,recall,f1-score,support,accuracy,roc_auc,ap,embedding
0,cello,irmas,irmas,0.30223880597014924,0.8617021276595744,0.4475138121546961,94.0,0.879951980792317,0.9396080342157977,0.5922021349589592,vggish
1,clarinet,irmas,irmas,0.24429223744292236,0.84251968503937,0.3787610619469027,127.0,0.7893157262905162,0.8945321893242877,0.5483587715419928,vggish
2,flute,irmas,irmas,0.3230240549828179,0.7768595041322314,0.4563106796116505,121.0,0.865546218487395,0.9072721923560407,0.6279013262077799,vggish
3,guitar,irmas,irmas,0.5902335456475584,0.8034682080924855,0.6805385556915545,346.0,0.8433373349339736,0.914453056577334,0.7957172213344217,vggish
4,organ,irmas,irmas,0.4219948849104859,0.9482758620689655,0.584070796460177,174.0,0.8589435774309724,0.9635758528242581,0.7474407674100789,vggish
...,...,...,...,...,...,...,...,...,...,...,...
75,piano,openmic,irmas,0.20903954802259886,0.8705882352941177,0.33712984054669703,170.0,0.6506602641056423,0.8811733249449513,0.6619703895398742,yamnet-lda
76,saxophone,openmic,irmas,0.22851153039832284,0.7266666666666667,0.34768740031897927,150.0,0.7545018007202882,0.8062181178540018,0.3545821389571038,yamnet-lda
77,trumpet,openmic,irmas,0.2618110236220472,0.8866666666666667,0.40425531914893614,150.0,0.7647058823529411,0.9101890941072999,0.5621971669640163,yamnet-lda
78,violin,openmic,irmas,0.2040072859744991,0.7832167832167832,0.3236994219653179,143.0,0.7190876350540216,0.8138381644619332,0.3224534901674157,yamnet-lda
